<a href="https://colab.research.google.com/github/wuwewij/bachelor_thesis/blob/main/%E7%BD%91%E7%BB%9C%E7%BB%93%E6%9E%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNetLike(nn.Module):
    def __init__(self):
        super(UNetLike, self).__init__()

        # Encoding path
        self.conv1_1 = nn.Conv2d(1, 16, 3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(16, 16, 3, stride=1, padding=1)
        self.conv2_1 = nn.Conv2d(32, 32, 3, stride=1, padding=1)
        self.conv3_1 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.conv4_1 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv4_2 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.conv5_1 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv5_2 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv6_1 = nn.Conv2d(256, 512, 3, stride=1, padding=1)

        # Decoding path
        self.conv6_2 = nn.Conv2d(512, 256, 3, stride=1, padding=1)
        self.conv7_1 = nn.Conv2d(384, 128, 3, stride=1, padding=1)
        self.conv8_1 = nn.Conv2d(192, 64, 3, stride=1, padding=1)
        self.conv9_1 = nn.Conv2d(96, 32, 3, stride=1, padding=1)
        self.conv10_1 = nn.Conv2d(48, 16, 3, stride=1, padding=1)
        self.conv10_2 = nn.Conv2d(16, 1, 3, stride=1, padding=1)

        self.bn = nn.BatchNorm2d(num_features=16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2, 2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        # Block 1
        x1_1 = self.relu(self.bn(self.conv1_1(x)))
        x1_2 = self.relu(self.bn(self.conv1_2(x1_1)))
        x1 = torch.cat((x1_1, x1_2), dim=1)
        x1 = self.maxpool(x1)

        # Block 2
        x2_1 = self.relu(self.bn(self.conv2_1(x1)))
        x2 = torch.cat((x2_1, x1), dim=1)
        x2 = self.maxpool(x2)

        # Block 3
        x3_1 = self.relu(self.bn(self.conv3_1(x2)))
        x3 = self.maxpool(x3_1)

        # Block 4
        x4_1 = self.relu(self.bn(self.conv4_1(x3)))
        x4_2 = self.relu(self.bn(self.conv4_2(x4_1)))
        x4 = self.maxpool(x4_2)

        # Block 5
        x5_1 = self.relu(self.bn(self.conv5_1(x4)))
        x5_2 = self.relu(self.bn(self.conv5_2(x5_1)))
        x5 = self.maxpool(x5_2)

        # Block 6
        x6_1 = self.relu(self.bn(self.conv6_1(x5)))
        x6_up = self.upsample(x6_1)
        x6_2 = self.relu(self.bn(self.conv6_2(x6_up)))
        x6 = torch.cat((x6_2, x4_2), dim=1)

        # Block 7
        x7_up = self.upsample(x6)
        x7_1 = self.relu(self.bn(self.conv7_1(x7_up)))
        x7 = torch.cat((x7_1, x3_1), dim=1)

        # Block 8
        x8_up = self.upsample(x7)
        x8_1 = self.relu(self.bn(self.conv8_1(x8_up)))

        # Block 9
        x9_up = self.upsample(x8)
        x9_1 = self.relu(self.bn(self.conv9_1(x9_up)))

        # Block 10
        x10_up = self.upsample(x9)
        x10_1 = self.relu(self.bn(self.conv10_1(x10_up)))
        x10_2 = self.conv10_2(x10_1)
        output = torch.sigmoid(x10_2)

        return output

# Example usage
model = UNetLike()
data = torch.randn(20, 1, 1024, 1024)  # Batch of 20 1024x1024 grayscale images
output = model(data)


RuntimeError: running_mean should contain 32 elements not 16